In [1]:
import json

In [3]:
def create_datasets(input_path, output_path):
    with open(input_path) as json_file:
        data = json.load(json_file)

    entries = []
    for i, entry in enumerate(data):
        if "qa" in entry:
            entries.append(create_type1(entry))
        elif "qa_0" in entry and "qa_1" in entry:
            entries.append(create_type2(entry))
        else:
            print(f"Found unusual entry at {i}")
        entries[-1]["index"] = i
    with open(output_path, "w") as out:
        out.write(json.dumps(entries))


def create_type1(entry):
    entry = entry["annotation"]
    step_list = entry["step_list"]
    context = " ".join([entry["amt_pre_text"], entry["amt_table"], entry["amt_post_text"]])
    is_op, answers = [], []
    for j, step in enumerate(step_list):
        if "Ask for number" in step:
            is_op.append(False)
            answers.append(step.split(" ")[-1])
        elif "(" in step:
            is_op.append(True)
            answers.append(step)
        else:
            print(f"Found unusual step {j} in entry {i}")
            break
    return {
        "context": context,
        "questions": entry["dialogue_break"],
        "is_op": is_op,
        "answers": answers,
        "exe_answers": entry["exe_ans_list"],
        "type": 1,
    }
        
def create_type2(entry):
    entry = entry["annotation"]
    step_list = entry["step_list"]
    context = " ".join([entry["amt_pre_text"], entry["amt_table"], entry["amt_post_text"]])
    dialogue_ori = entry["dialogue_break_ori"]
    questions, is_op, answers = [], [], []
    for j, (dialogue, step) in enumerate(zip(dialogue_ori, step_list)):
        if dialogue == "n/a":
            continue
        if "Ask for number" in step:
            questions.append(dialogue)
            is_op.append(False)
            answers.append(step.split(" ")[-1])
        elif "(" in step:
            questions.append(dialogue)
            is_op.append(True)
            answers.append(step)
        else:
            print(f"Found unusual step {j} in entry {i}")
            break
    return {
        "context": context,
        "questions": entry["dialogue_break"],
        "is_op": is_op,
        "answers": answers,
        "exe_answers": entry["exe_ans_list"],
        "type": 2,
    }

In [5]:
input_path = "../data/raw/train.json"
output_path = "../data/processed/train.json"

create_datasets(input_path, output_path)